# CartPole with PPO(Proximal Policy Optimization)
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import gym
import copy
import sys
sys.path.append('../material')
from utils import moving_average, discounted_reward, FIFO

from IPython.display import clear_output
from IPython.display import Video
from collections import namedtuple, deque

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """transition 저장"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)
np.random.seed(123)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
env = gym.make('CartPole-v0')
env.seed(123)
env._max_episode_steps=500

/home/chae/anaconda3/envs/python38/lib/python3.8/site-packages/gym/envs/registration.py:564: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/home/chae/anaconda3/envs/python38/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(
/home/chae/anaconda3/envs/python38/lib/python3.8/site-packages/gym/core.py:200: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


# PPO agent

In [6]:
class PPO_agent(nn.Module):
    def __init__(self, state_shape, n_actions):
        super(PPO_agent,self).__init__()
        '''
        입력변수
            state_shape: state 차원 -> [위치, 속도, 각도, 각속도]
            output_dim: actor 차원 -> [왼쪽, 오른쪽]
                        critic 차원 -> 1
            device : cpu, cuda device정보 
        N.N 구조
            2 - hidden layers, 64 nodes
            Activation function -> Relu
        '''
        self.state_shape = state_shape
        self.n_actions = n_actions
        
        self.seq = nn.Sequential(
            nn.Linear(self.state_shape,64), 
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
        )
        self.actor = nn.Linear(64,n_actions)
        self.critic = nn.Linear(64,1)

    def forward(self, state_t):
        policy = self.actor(self.seq(state_t))
        value = self.critic(self.seq(state_t))
        return policy, value

    def sample_actions(self,state_t):
        policy,_ = self.forward(state_t)
        policy = torch.squeeze(policy)
        softmax_policy = F.softmax(policy,dim=-1)
        action = torch.distributions.Categorical(softmax_policy).sample().item()
        return action

In [7]:
gamma=0.99
scheduler_gamma=0.995
scheduler_step=1000
epsilon = 1e-04
eps = 0.2
value_ratio=0.5

state = env.reset()
num_state = state.shape[0]
num_action = env.action_space.n
learning_rate = 1e-03
max_episode = 1000
update_per_episode=5

# Clipped Surrogate Objective
---
Objective function

$$

J(\theta) = \mathbb{E}_t[min(r_t(\theta)\hat{A}_t,clip(r_t(\theta),1-\epsilon,1+\epsilon)\hat{A}_t)]

$$

where,
$r_t(\theta) = \frac{\pi{(a_t \vert s_t)}}{\pi_{old}(a_t \vert s_t)}$

In [8]:
agent = PPO_agent(num_state,num_action).to(device)
old_agent = PPO_agent(num_state,num_action).to(device)
old_agent.load_state_dict(agent.state_dict())
optimizer = optim.Adam(agent.parameters(),lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=scheduler_step,gamma=scheduler_gamma)

In [9]:
def PPO_update(histories, num_update=3):
    old_agent.load_state_dict(agent.state_dict())
    states, actions, rewards, next_states, dones = histories
    #old_proba = torch.Tensor(policies).to(device).detach()
    torch_state = torch.Tensor(states).to(device)
    old_proba,_ = old_agent(torch_state)
    old_proba = F.softmax(old_proba,dim=-1)
    old_proba = torch.gather(old_proba, 1, torch.tensor(actions).unsqueeze(1).to(device)).detach()
    torch_next_state = torch.Tensor(next_states).to(device)
    for _ in range(num_update):
        policy, value = agent(torch_state)
        _, next_value = agent(torch_next_state)
        soft_policy = F.softmax(policy,dim=-1)
        #print(soft_policy.device, torch.Tensor(actions).unsqueeze(1).shape)
        cur_proba = torch.gather(soft_policy,1,torch.tensor(actions).unsqueeze(1).to(device))
        #old_proba = torch.Tensor(policies).to(device)
        ratio = torch.exp(torch.log(cur_proba)-torch.log(old_proba))
        
        returns =torch.Tensor(discounted_reward(rewards)).to(device).view(-1,1)
        #td_target = torch.Tensor(rewards).to(device) + gamma * next_value * (1-torch.Tensor(dones).to(device))
        td_target = returns + gamma * next_value*(1-torch.Tensor(dones).to(device))
        advantage = td_target - value
        advantage = (advantage - advantage.mean())/advantage.std()
        #td_delta = td_delta.detach().cpu().numpy()
        #advantage = torch.Tensor(discounted_reward(td_delta)).to(device)

        surr_1 = ratio*advantage
        surr_2 = torch.clip(ratio, 1-eps, 1+eps)*advantage
        clip_loss = torch.min(surr_1, surr_2)
        critic_loss = (td_target - value)**2
        entropy = -torch.sum(soft_policy*torch.log(soft_policy),dim=-1)
    
        # loss function build
        loss = torch.mean(-clip_loss + value_ratio*critic_loss - epsilon*entropy)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    return loss.item(), -clip_loss.mean().item(), critic_loss.mean().item(), entropy.mean().item()

In [10]:
reward_record, loss_record, critic_record, entropy_record = [], [], [], []
states, actions, rewards, next_states, dones = [],[],[],[],[]
expert = ReplayMemory(10000)

for ep in range(max_episode):
    total_reward = 0
    state = env.reset()
    
    while True:
        torch_state = torch.Tensor(state[None]).to(device)
        policy, value = agent(torch_state)
        #soft_policy = F.softmax(policy,dim=-1).squeeze()
        action =  agent.sample_actions(torch_state)
        next_state, reward, done, _ = env.step(action)
        
        action2 = torch.tensor([[action]], device=device)
        reward2 = torch.tensor([reward], device=device)
        next_state2 = torch.tensor(next_state, device=device).unsqueeze(0)
        
        expert.push(torch.tensor([state], device=device), action2, next_state2, reward2)

        states = FIFO(state, states)
        actions = FIFO(action, actions)
        rewards = FIFO(reward, rewards)
        next_states = FIFO(next_state, next_states)
        dones = FIFO(done, dones)

        if done:
            break

        state = next_state
        total_reward += reward
    history = (states, actions, rewards, next_states, dones)
    # Episode finish -> update agent model
    loss,clip,critic,entropy = PPO_update(history, update_per_episode)
    reward_record.append(total_reward)
    loss_record.append(loss) 
    critic_record.append(critic)
    entropy_record.append(entropy)

    if ep % 10 == 0:
        clear_output(True)
        print(f'{ep}번째 에피소드 결과')
        print(f'최근 10 에피소드 보상평균 = {np.mean(reward_record[-10:])}')

        # plt.figure(figsize=[12, 12])
        
        # plt.subplot(2,2,1)
        # plt.title("Total Reward")
        # plt.plot(reward_record)
        # plt.plot(moving_average(reward_record))
        # plt.grid()

        # plt.subplot(2,2,2)
        # plt.title("Loss trend")
        # plt.plot(loss_record)
        # plt.plot(moving_average(loss_record))
        # plt.grid()

        # plt.subplot(2,2,3)
        # plt.title("Advantage trend")
        # plt.plot(critic_record)
        # plt.plot(moving_average(critic_record))
        # plt.grid()
        
        # plt.subplot(2,2,4)
        # plt.title("Entropy trend")
        # plt.plot(entropy_record)
        # plt.plot(moving_average(entropy_record))
        # plt.grid()

        # plt.show()
    
   

990번째 에피소드 결과
최근 10 에피소드 보상평균 = 288.5


In [11]:
print(expert.memory[500])

import pickle

with open('expert.pkl', 'wb') as f:
    pickle.dump(expert, f)


expert = ReplayMemory(10000)

with open('expert.pkl', 'rb') as f:
    expert = pickle.load(f)

print(expert.memory[0])

Transition(state=tensor([[-0.1245,  0.0027,  0.0212, -0.2281]], device='cuda:0'), action=tensor([[1]], device='cuda:0'), next_state=tensor([[-0.1244,  0.1975,  0.0166, -0.5140]], device='cuda:0'), reward=tensor([1.], device='cuda:0'))
Transition(state=tensor([[ 0.0170,  0.2429, -0.0353, -0.3176]], device='cuda:0'), action=tensor([[0]], device='cuda:0'), next_state=tensor([[ 0.0219,  0.0483, -0.0417, -0.0363]], device='cuda:0'), reward=tensor([1.], device='cuda:0'))


# Evaluation 

In [12]:
# import gym.wrappers

# def record(initial_state,agent,env,vid):
#     state = initial_state
#     sum_rewards = 0

#     while True:
#         vid.capture_frame()
#         torch_state = torch.Tensor(state[None]).to(device)
#         policy, value = agent(torch_state)
#         soft_policy = F.softmax(policy,dim=-1).squeeze()
#         action =  agent.sample_actions(torch_state)

#         next_state,reward,done,_ = env.step(action)
        
#         state = next_state
#         sum_rewards += reward
#         if done:
#             break
#     vid.close()
#     return sum_rewards

# env = gym.make('CartPole-v0')
# env._max_episode_steps=600
# env.seed(123)
# vid = gym.wrappers.monitoring.video_recorder.VideoRecorder(env,path='./videos/CartPole/PPO_CartPole_video.mp4')
# vid.render_mode="rgb_array"

# state = env.reset()
# rewards = record(state,agent,env,vid)
# print(rewards)

In [13]:
# Video('./videos/CartPole/PPO_CartPole_video.mp4',embed=True,width=512, height=512)